# Análisis de los sentimientos

## Usando XGBoost en SageMaker

_Programa de Aprendizaje Profundo de Nanodegree | Despliegue_

---

En este ejemplo de uso del servicio SageMaker de Amazon, construiremos un modelo de árbol aleatorio para predecir el sentimiento de una reseña de película. Es posible que haya visto una versión de este ejemplo en una lección anterior, aunque se hubiera hecho usando el paquete sklearn. En cambio, utilizaremos el paquete XGBoost tal como nos lo proporciona Amazon.

## Instrucciones

Ya se le ha proporcionado un código de plantilla, y deberá implementar una funcionalidad adicional para completar con éxito este cuaderno. No necesitará modificar el código incluido más allá de lo solicitado. Las secciones que comienzan con '**TODO**' en el encabezado indican que debe completar o implementar alguna parte dentro de ellas. Se proporcionarán instrucciones para cada sección y los detalles de la implementación están marcados en el bloque de código con un comentario de `##TODO:... `. ¡Asegúrese de leer las instrucciones cuidadosamente!

Además de implementar el código, es posible que tenga que responder preguntas relacionadas con la tarea y su implementación. Cada sección donde responderá una pregunta está precedida por un encabezado '**Pregunta:**'. Lea atentamente cada pregunta y proporcione su respuesta debajo del encabezado '**Respuesta:**' editando la celda Markdown.

> **Nota**: las celdas de código y Markdown se pueden ejecutar usando el atajo de teclado **Shift + Enter**. Además, una celda se puede editar haciendo clic en ella (haciendo doble clic en las celdas Markdown) o presionando **Enter** mientras está resaltada.

## Paso 1: Descargar los datos

El conjunto de datos que vamos a utilizar es muy popular entre los investigadores del procesamiento del lenguaje natural, generalmente denominado [conjunto de datos de IMDb](http://ai.stanford.edu/~amaas/data/sentiment/). Consiste en reseñas de películas del sitio web [imdb.com](http://www.imdb.com/), cada una etiquetada como '**pos**itivo', si el crítico disfrutó la película, o '**neg**ativo'de lo contrario.

> Maas, Andrew L. y col. [Aprendizaje de vectores de palabras para el análisis de sentimientos] (http://ai.stanford.edu/~amaas/data/sentiment/). En _Procedimientos de la 49ª Reunión Anual de la Asociación de Lingüística Computacional: Tecnologías del Lenguaje Humano_. Asociación de Lingüística Computacional, 2011.

Comenzamos usando un poco de magia de Jupyter Notebook para descargar y extraer el conjunto de datos.

In [ ]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

## Paso 2: preparación de los datos

Los datos que hemos descargado se dividen en varios archivos, cada uno de los cuales contiene una única revisión. Será mucho más fácil avanzar si combinamos estos archivos individuales en dos archivos grandes, uno para capacitación y otro para prueba.

In [ ]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [ ]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [ ]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

In [ ]:
train_X[100]

## Paso 3: Procesando los datos

Ahora que tenemos nuestros conjuntos de datos de entrenamiento y prueba combinados y listos para usar, necesitamos comenzar a procesar los datos sin procesar en algo que nuestro algoritmo de aprendizaje automático pueda usar. Para comenzar, eliminamos cualquier formato html que pueda aparecer en las revisiones y realizamos un procesamiento estándar del lenguaje natural para homogeneizar los datos.

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [ ]:
import re
from bs4 import BeautifulSoup

def review_to_words(review):
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [ ]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [ ]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

### Extraer características de la bolsa de palabras

Para el modelo que implementaremos, en lugar de utilizar las revisiones directamente, vamos a transformar cada revisión en una representación característica de la Bolsa de palabras. Tenga en cuenta que 'en la naturaleza' solo tendremos acceso al conjunto de entrenamiento para que nuestro transformador solo pueda usar el conjunto de entrenamiento para construir una representación.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays

def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Fit a vectorizer to training documents and use it to transform them
        # NOTE: Training documents have already been preprocessed and tokenized into words;
        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x
        vectorizer = CountVectorizer(max_features=vocabulary_size,
                preprocessor=lambda x: x, tokenizer=lambda x: x)  # already preprocessed
        features_train = vectorizer.fit_transform(words_train).toarray()

        # Apply the same vectorizer to transform the test documents (ignore unknown words)
        features_test = vectorizer.transform(words_test).toarray()
        
        # NOTE: Remember to convert the features using .toarray() for a compact representation
        
        # Write to cache file for future runs (store vocabulary as well)
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    # Return both the extracted features as well as the vocabulary
    return features_train, features_test, vocabulary

In [ ]:
# Extract Bag of Words features for both training and test datasets
train_X, test_X, vocabulary = extract_BoW_features(train_X, test_X)

## Paso 4: Clasificación usando XGBoost

Ahora que hemos creado la representación de características de nuestros datos de entrenamiento (y pruebas), es hora de comenzar a configurar y usar el clasificador XGBoost provisto por SageMaker.

### Escribiendo el conjunto de datos

El clasificador XGBoost que utilizaremos requiere que el conjunto de datos se escriba en un archivo y se almacene con Amazon S3. Para hacer esto, comenzaremos dividiendo el conjunto de datos de entrenamiento en dos partes, los datos con los que entrenaremos al modelo y un conjunto de validación. Luego, escribiremos esos conjuntos de datos en un archivo y subiremos los archivos a S3. Además, escribiremos la entrada del conjunto de prueba en un archivo y subiremos el archivo a S3. Esto es para que podamos usar la funcionalidad de Transformación por lotes de SageMakers para probar nuestro modelo una vez que lo hayamos ajustado.

In [ ]:
import pandas as pd

val_X = pd.DataFrame(train_X[:10000])
train_X = pd.DataFrame(train_X[10000:])

val_y = pd.DataFrame(train_y[:10000])
train_y = pd.DataFrame(train_y[10000:])

test_y = pd.DataFrame(test_y)
test_X = pd.DataFrame(test_X)

La documentación para el algoritmo XGBoost en SageMaker requiere que los conjuntos de datos guardados no contengan encabezados o índice y que para los datos de capacitación y validación, la etiqueta debe aparecer primero para cada muestra.

Para obtener más información sobre este y otros algoritmos, la documentación para desarrolladores de SageMaker se puede encontrar en __[sitio web de Amazon.](Https://docs.aws.amazon.com/sagemaker/latest/dg/)__

In [ ]:
# First we make sure that the local directory in which we'd like to store the training and validation csv files exists.
data_dir = '../data/xgboost'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
# First, save the test data to test.csv in the data_dir directory. Note that we do not save the associated ground truth
# labels, instead we will use them later to compare with our model output.

# Solution:
# The test data shouldn't contain the ground truth labels as they are what the model is
# trying to predict. We will end up using them afterward to compare the predictions to.
# pd.concat([test_y, test_X], axis=1).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)
pd.DataFrame(test_X).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

pd.concat([val_y, val_X], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([train_y, train_X], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [ ]:
# To save a bit of memory we can set text_X, train_X, val_X, train_y and val_y to None.

train_X = val_X = train_y = val_y = None

### Carga de archivos de entrenamiento / validación a S3

El servicio S3 de Amazon nos permite almacenar archivos a los que pueden acceder tanto los modelos de entrenamiento incorporados como el modelo XGBoost que usaremos como los modelos personalizados como el que veremos un poco más adelante.

Para esta y la mayoría de las otras tareas que realizaremos con SageMaker, hay dos métodos que podríamos usar. El primero es utilizar la funcionalidad de bajo nivel de SageMaker que requiere conocer cada uno de los objetos involucrados en el entorno de SageMaker. El segundo es utilizar la funcionalidad de alto nivel en la que se han hecho ciertas elecciones en nombre del usuario. El enfoque de bajo nivel se beneficia al permitir al usuario una gran flexibilidad mientras que el enfoque de alto nivel hace que el desarrollo sea mucho más rápido. Para nuestros propósitos, optaremos por utilizar el enfoque de alto nivel, aunque usar el enfoque de bajo nivel es ciertamente una opción.

Recuerde el método `upload_data()`, que es un miembro del objeto que representa nuestra sesión actual de SageMaker. Lo que hace este método es cargar los datos en el depósito predeterminado (que se crea si no existe) en la ruta descrita por la variable key_prefix. Para ver esto por sí mismo, una vez que haya cargado los archivos de datos, vaya a la consola S3 y mire para ver dónde se han cargado los archivos.

Para obtener recursos adicionales, consulte la __[documentación de la API de SageMaker](http://sagemaker.readthedocs.io/en/latest/)__ y además la __[Guía del desarrollador de SageMaker.](Https: //docs.aws.amazon .com / sagemaker / latest / dg /)__

In [ ]:
import sagemaker

session = sagemaker.Session() # Store the current SageMaker session

# S3 prefix (which folder will we use)
prefix = 'sentiment-xgboost'

test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

### (TODO) Crear un modelo XGBoost sintonizado

Ahora que los datos se han cargado, es hora de crear el modelo XGBoost. Al igual que en el cuaderno Boston Housing, el primer paso es crear un objeto estimador que se utilizará como *base* de su trabajo de ajuste de hiperparámetros.

In [ ]:
from sagemaker import get_execution_role

# Our current execution role is require when creating the model as the training
# and inference code will need to access the model artifacts.
role = get_execution_role()

In [ ]:
# We need to retrieve the location of the container which is provided by Amazon for using XGBoost.
# As a matter of convenience, the training and inference code both use the same container.
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(session.boto_region_name, 'xgboost')

In [ ]:
# TODO: Create a SageMaker estimator using the container location determined in the previous cell.
#       It is recommended that you use a single training instance of type ml.m4.xlarge. It is also
#       recommended that you use 's3://{}/{}/output'.format(session.default_bucket(), prefix) as the
#       output path.

xgb = None

# Solution:
xgb = sagemaker.estimator.Estimator(container, # The location of the container we wish to use
                                    role,                                    # What is our current IAM Role
                                    train_instance_count=1,                  # How many compute instances
                                    train_instance_type='ml.m4.xlarge',      # What kind of compute instances
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                    sagemaker_session=session)

# TODO: Set the XGBoost hyperparameters in the xgb object. Don't forget that in this case we have a binary
#       label so we should be using the 'binary:logistic' objective.

# Solution:
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)

### (TODO) Cree el sintonizador de hiperparámetro

Ahora que se ha configurado el estimador base, necesitamos construir un objeto sintonizador de hiperparámetro que usaremos para solicitar a SageMaker que construya un trabajo de ajuste de hiperparámetro.

**Nota:** La capacitación de un modelo XGBoost de análisis de sentimiento único lleva más tiempo que la capacitación de un modelo XGBoost de Boston Housing, por lo que si no desea que el trabajo de ajuste de hiperparámetro demore demasiado, asegúrese de no establecer el número total de modelos (trabajos) demasiado alto.

In [ ]:
# First, make sure to import the relevant objects used to construct the tuner
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

# TODO: Create the hyperparameter tuner object

xgb_hyperparameter_tuner = None

# Solution:

xgb_hyperparameter_tuner = HyperparameterTuner(estimator = xgb, # The estimator object to use as the basis for the training jobs.
                                               objective_metric_name = 'validation:rmse', # The metric used to compare trained models.
                                               objective_type = 'Minimize', # Whether we wish to minimize or maximize the metric.
                                               max_jobs = 6, # The total number of models to train
                                               max_parallel_jobs = 3, # The number of models to train in parallel
                                               hyperparameter_ranges = {
                                                    'max_depth': IntegerParameter(3, 12),
                                                    'eta'      : ContinuousParameter(0.05, 0.5),
                                                    'min_child_weight': IntegerParameter(2, 8),
                                                    'subsample': ContinuousParameter(0.5, 0.9),
                                                    'gamma': ContinuousParameter(0, 10),
                                               })

### Montar el sintonizador de hiperparámetro

Ahora que se ha construido el objeto de sintonizador de hiperparámetro, es hora de ajustar los distintos modelos y encontrar el modelo con mejor rendimiento.

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

In [ ]:
xgb_hyperparameter_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

Recuerde que el trabajo de ajuste se construye y ejecuta en segundo plano, por lo que si queremos ver el progreso de nuestro trabajo de capacitación, debemos llamar al método `wait()`.

In [ ]:
xgb_hyperparameter_tuner.wait()

### (TODO) Prueba del modelo

Ahora que hemos ejecutado nuestro trabajo de ajuste de hiperparámetros, es hora de ver qué tan bien se desempeña realmente el modelo con mejor rendimiento. Para hacer esto, utilizaremos la funcionalidad de transformación por lotes de SageMaker. Batch Transform es una forma conveniente de realizar inferencia en un gran conjunto de datos de una manera que no es en tiempo real. Es decir, no necesariamente necesitamos usar los resultados de nuestro modelo de inmediato y en su lugar podemos realizar inferencias en una gran cantidad de muestras. Un ejemplo de esto en la industria podría ser realizar un informe de fin de mes. Este método de inferencia también puede sernos útil, ya que significa realizar inferencia en todo nuestro conjunto de pruebas.

Recuerde que para crear un objeto transformador para realizar el trabajo de transformación por lotes, necesitamos un objeto estimador capacitado. Podemos hacerlo utilizando el método `attach()`, creando un objeto estimador que se adjunta al trabajo mejor capacitado.

In [ ]:
# TODO: Create a new estimator object attached to the best training job found during hyperparameter tuning

xgb_attached = None

# Solution:

xgb_attached = sagemaker.estimator.Estimator.attach(xgb_hyperparameter_tuner.best_training_job())

Ahora que tenemos un objeto estimador adjunto al trabajo de entrenamiento correcto, podemos proceder como lo haríamos normalmente y crear un objeto transformador.

In [ ]:
# TODO: Create a transformer object from the attached estimator. Using an instance count of 1 and an instance type of ml.m4.xlarge
#       should be more than enough.

xgb_transformer = None

# Solution:
xgb_transformer = xgb_attached.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')

A continuación, realizamos el trabajo de transformación. Al hacerlo, debemos asegurarnos de especificar el tipo de datos que estamos enviando para que se serialicen correctamente en segundo plano. En nuestro caso, proporcionamos a nuestro modelo datos csv, por lo que especificamos `text/csv`. Además, si los datos de prueba que proporcionamos son demasiado grandes para procesarlos a la vez, entonces debemos especificar cómo se debe dividir el archivo de datos. Como cada línea es una entrada única en nuestro conjunto de datos, le decimos a SageMaker que puede dividir la entrada en cada línea.

In [ ]:
# TODO: Start the transform job. Make sure to specify the content type and the split type of the test data.
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')

Actualmente el trabajo de transformación se está ejecutando pero lo está haciendo en segundo plano. Como deseamos esperar hasta que se complete el trabajo de transformación y nos gustaría un poco de retroalimentación, podemos ejecutar el método `wait()`.

In [ ]:
xgb_transformer.wait()

Ahora el trabajo de transformación se ha ejecutado y el resultado, el sentimiento estimado de cada revisión, se ha guardado en S3. Como preferimos trabajar en este archivo localmente, podemos realizar un poco de magia de cuaderno para copiar el archivo al `data_dir`.

In [ ]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

El último paso ahora es leer la salida de nuestro modelo, convertir la salida a algo un poco más utilizable, en este caso queremos que el sentimiento sea `1` (positivo) o` 0` (negativo), y luego comparar con las etiquetas de verdad básicas.

In [ ]:
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

## Opcional: limpiar

La instancia de notebook predeterminada en SageMaker no tiene mucho espacio en disco disponible. A medida que continúe completando y ejecutando cuadernos, eventualmente llenará este espacio en disco, lo que generará errores que pueden ser difíciles de diagnosticar. Una vez que haya terminado de usar un cuaderno, es una buena idea eliminar los archivos que creó en el camino. Por supuesto, puede hacerlo desde la terminal o desde el hub del portátil si lo desea. La celda a continuación contiene algunos comandos para limpiar los archivos creados desde el cuaderno.

In [ ]:
# First we will remove all of the files contained in the data_dir directory
!rm $data_dir/*

# And then we delete the directory itself
!rmdir $data_dir

# Similarly we will remove the files in the cache_dir directory and the directory itself
!rm $cache_dir/*
!rmdir $cache_dir